In [1]:
import cv2
import numpy as np
import random as rng
from utils import *

# https://stackoverflow.com/questions/59182827/how-to-get-the-cells-of-a-sudoku-grid-with-opencv
# https://stackoverflow.com/questions/10196198/how-to-remove-convexity-defects-in-a-sudoku-square
# https://docs.opencv.org/3.4/dd/dd7/tutorial_morph_lines_detection.html
# https://stackoverflow.com/questions/60396925/how-to-find-the-number-of-rows-and-columns-in-a-table-image-with-python-opencv
# https://golsteyn.com/writing/sudoku

rng.seed(12345)

def show_wait_destroy(winname, img):
    cv2.imshow(winname, img)
    cv2.moveWindow(winname, 500, 0)
    cv2.waitKey(0)
    cv2.destroyWindow(winname)

def morphological_closing(gray):
    # https://stackoverflow.com/questions/10561222/how-do-i-equalize-contrast-brightness-of-images-using-opencv
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11)) # originally 11,11
    close = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel1)
    div = np.float32(gray)/(close)
    output = np.uint8(cv2.normalize(div, div, 0, 255, cv2.NORM_MINMAX))
    return output

def extract_largest_contour(input, output):
    mask = np.zeros((input.shape), np.uint8)
    contours, hierarchy = cv2.findContours(input, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # find the biggest contour
    max_area = 0
    best_cnt = None
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            best_cnt = cnt

    cv2.drawContours(mask, [best_cnt], 0, 255, -1) # full color (255) inverted
    cv2.drawContours(mask, [best_cnt], 0, 0, 2)    # no color (0) thickness 2

    # increase mask size so we don't cut away the lines when bitwising
    mask = cv2.dilate(mask, None, iterations=3)

    output = cv2.bitwise_and(output, mask)
    return output

def extractVerticalLines(input):
    # Finding Vertical lines
    kernel1X = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 10))

    dx = cv2.Sobel(input, cv2.CV_8U, dx=2, dy=0) # originally CV_16S
    dx = cv2.convertScaleAbs(dx)
    cv2.normalize(dx, dx, 0, 255, cv2.NORM_MINMAX)
    ret, close = cv2.threshold(dx, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    close = cv2.morphologyEx(close, cv2.MORPH_DILATE, kernel1X, iterations = 1)

    contours, hierarchy = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if h/w > 20: # originally 5
            cv2.drawContours(close, [cnt], 0, 255, -1)  # full color (255)
        else:
            cv2.drawContours(close, [cnt], 0, 0, -1)    # no color (0)

    close = cv2.morphologyEx(close, cv2.MORPH_CLOSE, None, iterations = 2)
    closeX = close.copy()
    return closeX

def extractHorizontalLines(input):
    # Finding Horizontal Lines
    kernel1Y = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 2))
    dy = cv2.Sobel(input, cv2.CV_8U, dx=0, dy=2) # originally CV_16S
    dy = cv2.convertScaleAbs(dy)
    cv2.normalize(dy, dy, 0, 255, cv2.NORM_MINMAX)
    ret, close = cv2.threshold(dy, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    close = cv2.morphologyEx(close, cv2.MORPH_DILATE, kernel1Y)

    contours, hierarchy = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w/h > 20: # originally 5
            cv2.drawContours(close, [cnt], 0, 255, -1)  # full color (255)
        else:
            cv2.drawContours(close, [cnt], 0, 0, -1)   # no color (0)

    close = cv2.morphologyEx(close, cv2.MORPH_DILATE, None, iterations = 2)
    closeY = close.copy()
    return closeY

def get_cropped_image(image, x, y, w, h):
    cropped_image = image[ y:y+h, x:x+w ]
    return cropped_image

def get_ROI(image, horizontal, vertical, left_line_index, right_line_index, top_line_index, bottom_line_index, offset=4):
    # https://levelup.gitconnected.com/text-extraction-from-a-table-image-using-pytesseract-and-opencv-3342870691ae
    x1 = vertical[left_line_index][2] + offset
    y1 = horizontal[top_line_index][3] + offset
    x2 = vertical[right_line_index][2] - offset
    y2 = horizontal[bottom_line_index][3] - offset
    
    w = x2 - x1
    h = y2 - y1
    
    cropped_image = get_cropped_image(image, x1, y1, w, h)
    
    return cropped_image, (x1, y1, w, h)

def writeArrayToDisk(arr, out = 'array_out.txt'):
    dim = arr.ndim 
        
    with open(out, 'w') as outfile:    
        outfile.write('# Array shape: {0}\n'.format(arr.shape))
        
        if dim == 1 or dim == 2:
            np.savetxt(outfile, arr, fmt='%10.3f')
            # output as CSV
            # np.savetxt(outfile, arr, delimiter=",", fmt="%10.5f")

        elif dim == 3:
            for i, arr2d in enumerate(arr):
                outfile.write('# {0}-th channel\n'.format(i))
                np.savetxt(outfile, arr2d, fmt='%10.3f')
                
        elif dim == 4:
            for j, arr3d in enumerate(arr):
                outfile.write('\n# {0}-th Image\n'.format(j))
                for i, arr2d in enumerate(arr3d):
                    outfile.write('# {0}-th channel\n'.format(i))
                    np.savetxt(outfile, arr2d, fmt='%10.3f')

        else:
            print("Out of dimension!")

# ####################################

image = cv2.imread('crossword-1.png')

show_wait_destroy("raw image", image)

# Transform source image to gray if it is not already
if len(image.shape) != 2:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
else:
    gray = image

show_wait_destroy("gray image", gray)

grayOriginal = gray.copy();

# gray = cv2.GaussianBlur(gray, (3,3), 0)
# show_wait_destroy("gray blurred", gray)

# gray = morphological_closing(gray)
# show_wait_destroy("gray closed", gray)

# gray = maximizeContrast(gray)
# show_wait_destroy("maximized contrast", gray)

# using a big blocksize seem to work well (blocksize = 51, c = 11)
thresh = cv2.adaptiveThreshold( 
    gray,
    maxValue=255.0,
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    thresholdType=cv2.THRESH_BINARY_INV,
    blockSize=51,
    C=11
)

# apply adaptiveThreshold at the bitwise_not of gray, notice the ~ symbol
# thresh = cv2.adaptiveThreshold( 
#     ~gray,
#     maxValue=255.0,
#     adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C,
#     thresholdType=cv2.THRESH_BINARY,
#     blockSize=51,
#     C=-11
# )

# Show binary image
show_wait_destroy("thresh", thresh)

# Filter out all numbers and noise to isolate only boxes
# seem not to be needed, but keep it anyway
cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area < 5000:
        cv2.drawContours(thresh, [c], -1, (0,0,0), -1)
show_wait_destroy("thresh2", thresh)

# extract largest contour
thresh = extract_largest_contour(thresh, thresh)
show_wait_destroy("largest_contour", thresh)

# Fix horizontal and vertical lines (thickening)
# vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,5))
# thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, vertical_kernel, iterations=4)

# horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,1))
# thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, horizontal_kernel, iterations=4)

# show_wait_destroy("thresh3", thresh)

# Find number of rows
horizontal_mask = np.zeros((gray.shape), np.uint8)
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,1))
horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=4)
cnts = cv2.findContours(horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
rows = 0
for c in cnts:
    area = cv2.contourArea(c)
    if area > 5000:
        # cv2.drawContours(horizontal_mask, [c], -1, (0,255,255), 5)
        cv2.drawContours(horizontal_mask, [c], 0, 255, -1)  # full color (255)
        rows += 1

show_wait_destroy("horizontal_mask", horizontal_mask)

# Find number of columns
vertical_mask = np.zeros((gray.shape), np.uint8)
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,5))
vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=4)
cnts = cv2.findContours(vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
columns = 0
for c in cnts:
    area = cv2.contourArea(c)
    if area > 5000:
        # cv2.drawContours(vertical_mask, [c], -1, (0,255,0), 5)
        cv2.drawContours(vertical_mask, [c], 0, 255, -1)  # full color (255)
        columns += 1

show_wait_destroy("vertical_mask", vertical_mask)

# Combine masks 
table_mask = cv2.bitwise_or(horizontal_mask, vertical_mask)
show_wait_destroy("table_mask", table_mask)

# 5. Finding Grid Points ( intersection of these two gives dots )
intersections = cv2.bitwise_and(horizontal_mask, vertical_mask)
show_wait_destroy("intersections", intersections)

# 6. Correcting the defects
# find centroids and sort
contours, hierarchy = cv2.findContours(intersections, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
# cv2.drawContours(image, contours, -1, (255, 0, 0), 3) # DRAW ALL DETECTED CONTOURS (using -1)

centroids = []
# for cnt in contours:
for i in range(len(contours)):
    mom = cv2.moments(contours[i])

    # add 1e-5 to avoid division by zero
    (x, y) = int(mom['m10']/mom['m00'] + 1e-5), int(mom['m01']/mom['m00'] + 1e-5)

    # debug draw contour with random color
    # color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    # cv2.drawContours(image, contours, i, color, 5)

    cv2.circle(image, (x, y), 10, (0, 255, 0), -1)
    centroids.append((x, y))

show_wait_destroy("centroids", image)

# text green
textColor=(255, 0, 0)

# points blue
pointColor=(0, 255, 0)

labeled=image.copy()
for index, pt in enumerate(centroids):
    cv2.putText(labeled, str(index), (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, textColor, 2)
    cv2.circle(labeled, (int(pt[0]), int(pt[1])), 5, pointColor, -1)

show_wait_destroy("labeled", labeled)

# test grid is 27 x 21 = 567 cells
print('Rows:', rows - 1)        # 27
print('Columns:', columns - 1)  # 21

# sorting
centroids = np.array(centroids, dtype = np.float32)
writeArrayToDisk(centroids, 'temp/centroids.txt')

# print (len(centroids)) # = 616 
# 21 x 27 = 567
# (21 + 1) x (27 + 1) = 616
c = centroids.reshape((len(centroids), 2)) # convert from a list of (x, y) to an array with two columns
writeArrayToDisk(c, 'temp/c.txt')

c2 = c[np.argsort(c[:, 1])] # sort by second column, i.e. y
writeArrayToDisk(c2, 'temp/c2.txt')

# divide the array into chunks of rows
# vstack stack arrays in sequence vertically (row wise).
# for a 9 x 9 grid
# b = np.vstack([c2[i*10:(i+1)*10][np.argsort(c2[i*10:(i+1)*10, 0])] for i in range(10)])
# for a 21 x 27 grid
# b = np.vstack([c2[i*22:(i+1)*22][np.argsort(c2[i*22:(i+1)*22, 0])] for i in range(28)])
# rows and columns are actual lines not actual rows and columns, so we don't have to subtract 1 to use them
b = np.vstack([c2[i*columns:(i+1)*columns][np.argsort(c2[i*columns:(i+1)*columns, 0])] for i in range(rows)])
writeArrayToDisk(b, 'temp/b.txt')

labeled_in_order=image.copy()
for index, pt in enumerate(b):
    cv2.putText(labeled_in_order, str(index), (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, textColor, 2)
    cv2.circle(labeled_in_order, (int(pt[0]), int(pt[1])), 5, pointColor, -1)

show_wait_destroy("labeled in order", labeled_in_order)
cv2.imwrite("temp/labeled_in_order.png", labeled_in_order)

# create final image using a mesh
size = 50 # pixel width and height of each cell
res2 = cv2.cvtColor(grayOriginal, cv2.COLOR_GRAY2BGR)
output = np.zeros((size*(rows-1), size*(columns-1), 3), np.uint8) # output a rgb color image

bm = b.reshape((rows, columns, 2)) # convert into a multidimensional array
writeArrayToDisk(bm, 'temp/bm.txt')

# debug to file
# f = open("temp/output.txt", "w")
for index, pt in enumerate(b):
    ri = int(index/columns) # row index
    ci = index%columns # column index  
    if ci != (columns-1) and ri != (rows-1):
        # define the src polygon
        src = bm[ri:ri+2, ci:ci+2, :].reshape((4, 2)) # extract 4 points at a time
        
        # f.write('index: %s  ri: %s  ci: %s\n' % (index, ri, ci))
        # f.write('src: (%s, %s, %s, %s)\n' % (src[0], src[1], src[2], src[3]))
        
        # [0, 0], [width, 0], [0, height], [width, height]
        # define the destination square
        dst = np.array( [ 
                [ci*size, ri*size], 
                [(ci+1)*size-1, ri*size], 
                [ci*size, (ri+1)*size-1], 
                [(ci+1)*size-1, (ri+1)*size-1] 
            ], np.float32)

        # f.write('dst: (%s, %s, %s, %s)\n\n' % (dst[0], dst[1], dst[2], dst[3]))

        retval = cv2.getPerspectiveTransform(src, dst)
        warp = cv2.warpPerspective(res2, retval, (size*(columns-1), size*(rows-1)))
        output[ri*size:(ri+1)*size-1, ci*size:(ci+1)*size-1] = warp[ri*size:(ri+1)*size-1, ci*size:(ci+1)*size-1].copy()

        # cell = output[ri*size:(ri+1)*size-1, ci*size:(ci+1)*size-1]
        # cv2.imwrite("temp/cell%s.png" % index, output)

    else:
        # f.write('IGNORED index: %s  ri: %s  ci: %s\n\n' % (index, ri, ci))
        pass
        
# f.close()

show_wait_destroy("output", output)
cv2.imwrite("temp/output.png", output)


Rows: 27
Columns: 21


True